In [5]:
import numpy as np

# ----- Activation Functions -----
def relu(x):
    """ReLU activation."""
    return np.maximum(0, x)

def leaky_relu(x, alpha=0.01):
    """Leaky ReLU activation."""
    return np.where(x > 0, x, alpha * x)

# ----- Convolution Utilities -----
def conv2d_single_channel(X, W):
    """Compute 2D convolution (no stride, valid region)."""
    H, W_in = X.shape
    kH, kW = W.shape
    out = np.zeros((H - kH + 1, W_in - kW + 1))
    for i in range(out.shape[0]):
        for j in range(out.shape[1]):
            out[i, j] = np.sum(X[i:i+kH, j:j+kW] * W)
    return out

def zero_pad(X, pad=1):
    """Zero pad a 2D matrix."""
    return np.pad(X, ((pad, pad), (pad, pad)), mode='constant', constant_values=0)

# ----- Multi-channel Convolution -----
def conv2d_multichannel_verbose(X, W, bias=0, pad=1, activation="relu", alpha=0.01):
    """
    Compute convolution for 3-channel input X (H x W x C)
    and 3-channel filter W (kH x kW x C), printing each step.

    Parameters
    ----------
    X : np.ndarray
        Input image of shape (H, W, C)
    W : np.ndarray
        Filter kernel of shape (kH, kW, C)
    bias : float or np.ndarray
        Bias term to add after convolution
    pad : int
        Number of zero-padding pixels
    activation : str
        Activation function to apply ('relu', 'leaky_relu', or 'none')
    alpha : float
        Negative slope for Leaky ReLU
    """
    H, W_in, C = X.shape
    kH, kW, Cw = W.shape
    assert C == Cw, "Channel mismatch!"

    # Step 1: Zero-pad each channel
    X_padded = [zero_pad(X[:, :, c], pad) for c in range(C)]
    for c in range(2):
        print(f"\n=== Channel {c} Padded Input (X_tilde[:,:,{c}]) ===")
        print(X_padded[c])
        print(f"W[:,:,{c}] =\n{W[:,:,c]}")

    # Step 2: Convolve each channel
    conv_results = []
    for c in range(C):
        conv_c = conv2d_single_channel(X_padded[c], W[:, :, c])
        conv_results.append(conv_c)
        print(f"\n>>> Convolution result for channel {c} (X_tilde * W):")
        print(conv_c)

    # Step 3: Sum over channels
    summed = sum(conv_results)
    print("\n=== Summed convolution result before bias and activation ===")
    print(summed)

    # Step 4: Add bias
    summed_with_bias = summed + bias
    print("\n=== After adding bias ===")
    print(summed_with_bias)

    # Step 5: Apply chosen activation
    if activation == "relu":
        out = relu(summed_with_bias)
        print("\n=== After ReLU ===")
        print(out)
    elif activation == "leaky_relu":
        out = leaky_relu(summed_with_bias, alpha)
        print(f"\n=== After Leaky ReLU (alpha={alpha}) ===")
        print(out)
    elif activation == "none":
        out = summed_with_bias
        print("\n=== No activation applied ===")
        print(out)
    else:
        raise ValueError("Unknown activation: choose from 'relu', 'leaky_relu', or 'none'")

    return out


In [7]:
# ==== Example Matrices (from your 2021.1) ====
X0 = np.array([[2, 2, 0, 0],
               [0, 0, 2, 0],
               [0, 1, 1, 1],
               [1, 1, 0, 1]])
X1 = np.array([[0, 2, 2, 1],
               [0, 0, 2, 1],
               [1, 0, 1, 2],
               [1, 1, 0, 0]])
X2 = np.array([[2, 0, 0, 0],
               [2, 1, 1, 0],
               [2, 1, 0, 0],
               [1, 0, 0, 0]])
W0 = np.array([[-1, 0, 0],
               [0, 1, 0],
               [0, 0, -1]])
W1 = np.array([[-1, -1, 0],
               [1,  0,  1],
               [0, -1, -1]])
W2 = np.array([[0, -1, 1],
               [0,  2, 0],
               [1, -1, 0]])

X = np.stack([X0, X1, X2], axis=2)
W = np.stack([W0, W1, W2], axis=2)
bias = np.ones((4, 4))  # 1_{4x4}

# ==== Run Verbose Convolution ====
Y = conv2d_multichannel_verbose(X, W, bias=bias)



=== Channel 0 Padded Input (X_tilde[:,:,0]) ===
[[0 0 0 0 0 0]
 [0 2 2 0 0 0]
 [0 0 0 2 0 0]
 [0 0 1 1 1 0]
 [0 1 1 0 1 0]
 [0 0 0 0 0 0]]
W[:,:,0] =
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]

=== Channel 1 Padded Input (X_tilde[:,:,1]) ===
[[0 0 0 0 0 0]
 [0 0 2 2 1 0]
 [0 0 0 2 1 0]
 [0 1 0 1 2 0]
 [0 1 1 0 0 0]
 [0 0 0 0 0 0]]
W[:,:,1] =
[[-1 -1  0]
 [ 1  0  1]
 [ 0 -1 -1]]

=== Channel 2 Padded Input (X_tilde[:,:,2]) ===
[[0 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 2 1 1 0 0]
 [0 2 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 0]]
W[:,:,2] =
[[ 0 -1  1]
 [ 0  2  0]
 [ 1 -1  0]]

>>> Convolution result for channel 0 (X_tilde * W):
[[ 2.  0.  0.  0.]
 [-1. -3. -1.  0.]
 [-1.  1.  0. -1.]
 [ 1.  1. -1.  0.]]

>>> Convolution result for channel 1 (X_tilde * W):
[[ 2.  0.  0.  1.]
 [-1. -1. -6. -3.]
 [-2.  1.  0. -2.]
 [ 0.  0.  0. -3.]]

>>> Convolution result for channel 2 (X_tilde * W):
[[ 2.  1.  0.  1.]
 [ 0.  3.  3.  0.]
 [ 2.  3. -1.  0.]
 [ 1. -1.  0.  0.]]

=== Summed convolution result before bias and

In [8]:
# ==== Example Matrices (from your 2021.1) ====
X0 = np.array([[2, 1, 0, 0],
               [0, 0, 2, 1],
               [0, 2, 0, 1],
               [2, 1, 0, 1]])
X1 = np.array([[2, 2, 0, 0],
               [0, 0, 2, 1],
               [0, 0, 2, 0],
               [0, 1, 0, 1]])
X2 = np.array([[2, 1, 0, 0],
               [0, 0, 2, 1],
               [2, 0, 0, 0],
               [0, 1, 0, 1]])
W0 = np.array([[1, 0, 0],
               [0, -2, 0],
               [0, 0, -1]])
W1 = np.array([[1, 2, 0],
               [2,  0, -1],
               [0, -1, 1]])
W2 = np.array([[0, 0, -2],
               [0, 1, 2],
               [-2, 2, 0]])

X = np.stack([X0, X1, X2], axis=2)
W = np.stack([W0, W1, W2], axis=2)
bias = 2* np.ones((4, 4))  # 1_{4x4}

# ==== Run Verbose Convolution ====
Y = conv2d_multichannel_verbose(X, W, bias=bias)



=== Channel 0 Padded Input (X_tilde[:,:,0]) ===
[[0 0 0 0 0 0]
 [0 2 1 0 0 0]
 [0 0 0 2 1 0]
 [0 0 2 0 1 0]
 [0 2 1 0 1 0]
 [0 0 0 0 0 0]]
W[:,:,0] =
[[ 1  0  0]
 [ 0 -2  0]
 [ 0  0 -1]]

=== Channel 1 Padded Input (X_tilde[:,:,1]) ===
[[0 0 0 0 0 0]
 [0 2 2 0 0 0]
 [0 0 0 2 1 0]
 [0 0 0 2 0 0]
 [0 0 1 0 1 0]
 [0 0 0 0 0 0]]
W[:,:,1] =
[[ 1  2  0]
 [ 2  0 -1]
 [ 0 -1  1]]

=== Channel 2 Padded Input (X_tilde[:,:,2]) ===
[[0 0 0 0 0 0]
 [0 2 1 0 0 0]
 [0 0 0 2 1 0]
 [0 2 0 0 0 0]
 [0 0 1 0 1 0]
 [0 0 0 0 0 0]]
W[:,:,2] =
[[ 0  0 -2]
 [ 0  1  2]
 [-2  2  0]]

>>> Convolution result for channel 0 (X_tilde * W):
[[-4. -4. -1.  0.]
 [-2.  2. -4. -2.]
 [-1. -4. -1.  0.]
 [-4. -2.  2. -2.]]

>>> Convolution result for channel 1 (X_tilde * W):
[[-2.  6.  3. -1.]
 [ 4.  6. -1.  4.]
 [ 1. -3.  5.  7.]
 [-1.  0.  5.  2.]]

>>> Convolution result for channel 2 (X_tilde * W):
[[ 4.  1.  4. -2.]
 [ 2.  0.  4.  1.]
 [ 2. -2. -4.  2.]
 [ 2.  1.  2.  1.]]

=== Summed convolution result before bias and

In [6]:
# ==== Example Matrices (from your 2021.1) ====
X0 = np.array([[2, 0, 1, 2],
               [1, 3, 0, 1],
               [0, 1, 2, 0],
               [1, 1, 0, 2]])
X1 = np.array([[1, 1, 2, 0],
               [0, 0, 1, 0],
               [0, 1, 2, 1],
               [0, 0, 2, 1]])
W0 = np.array([[1, 0, -1],
               [0, 1, 0],
               [-1, 0, 1]])
W1 = np.array([[-1, 0, 1],
               [0,  1, 0],
               [1, 0, -1]])


X = np.stack([X0, X1], axis=2)
W = np.stack([W0, W1], axis=2)
bias = (-1)* np.ones((4, 4))  # 1_{4x4}

# ==== Run Verbose Convolution ====
Y = conv2d_multichannel_verbose(X, W, bias=bias, activation="leaky_relu", alpha=0.1)



=== Channel 0 Padded Input (X_tilde[:,:,0]) ===
[[0 0 0 0 0 0]
 [0 2 0 1 2 0]
 [0 1 3 0 1 0]
 [0 0 1 2 0 0]
 [0 1 1 0 2 0]
 [0 0 0 0 0 0]]
W[:,:,0] =
[[ 1  0 -1]
 [ 0  1  0]
 [-1  0  1]]

=== Channel 1 Padded Input (X_tilde[:,:,1]) ===
[[0 0 0 0 0 0]
 [0 1 1 2 0 0]
 [0 0 0 1 0 0]
 [0 0 1 2 1 0]
 [0 0 0 2 1 0]
 [0 0 0 0 0 0]]
W[:,:,1] =
[[-1  0  1]
 [ 0  1  0]
 [ 1  0 -1]]

>>> Convolution result for channel 0 (X_tilde * W):
[[ 5. -1. -1.  2.]
 [ 2.  6. -3.  0.]
 [-2.  1.  5.  0.]
 [ 0. -1.  1.  4.]]

>>> Convolution result for channel 1 (X_tilde * W):
[[ 1.  0.  2.  1.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  2.]
 [ 1.  2.  2. -1.]]

=== Summed convolution result before bias and activation ===
[[ 6. -1.  1.  3.]
 [ 2.  5. -3.  0.]
 [-2.  1.  6.  2.]
 [ 1.  1.  3.  3.]]

=== After adding bias ===
[[ 5. -2.  0.  2.]
 [ 1.  4. -4. -1.]
 [-3.  0.  5.  1.]
 [ 0.  0.  2.  2.]]

=== After Leaky ReLU (alpha=0.1) ===
[[ 5.  -0.2  0.   2. ]
 [ 1.   4.  -0.4 -0.1]
 [-0.3  0.   5.   1. ]
 [ 0.   0.   2